In [ ]:
"""
This notebook is used to generate visualizations for user study based on Chen et al. Planning with Trust for Human-Robot Collaboration

The visualization is done using PyBullet, rougphly following the methodology described in this blog post (we don't use the OpenAI gym
though since it is not interactive): https://www.etedal.net/2020/04/pybullet-panda.html

Documentation on PyBullet can be found here: https://pybullet.org/wordpress/index.php/forum-2/

ChatGPT was used for some initial code generation to learn how PyBullet works.

The last cell will allow you to download the video files. It initially saves the video to a Google folder, so you may need to check the file
names before downloading.
"""

In [ ]:
!pip3 install pybullet --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pybullet: filename=pybullet-3.2.7-cp312-cp312-linux_x86_64.whl size=99873170 sha256=0a30c9b3b60d92f3fc9c53a615c77acc38c7a4caf46527bdf0f2f3396247cb8e
  Stored in directory: /root/.cache/pip/wheels/72/95/1d/b336e5ee612ae9a019bfff4dc0bedd100ee6f0570db205fdf8
Successfully built pybullet


In [ ]:
import os
import math
from pathlib import Path

import imageio
import numpy as np
import pybullet as p
import pybullet_data

FPS = 30
OUT_DIR = Path("videos")
OUT_DIR.mkdir(exist_ok=True)

URDF_ROOT = pybullet_data.getDataPath()
PANDA_EE_LINK = 11

In [ ]:
#define objects
URDF_CHOICES = {
    "bottle": "000/000.urdf",
    "can":    "001/001.urdf",
    "glass":  "002/002.urdf",
}


OBJ_HEIGHTS = {
    "bottle": 0.22,
    "can":    0.12,
    "glass":  0.18,
}

# Visual target on the table
PLACE_POS = [0.45, -0.25, 0.0]

# Note:

  # +x points away from the robot (toward the 5-object row)

  # −x moves the marker toward the robot base

  # +y shifts the marker to the robot’s left

  # −y shifts the marker to the robot’s right

In [ ]:
def connect_headless():
    cid = p.connect(p.DIRECT)
    p.setAdditionalSearchPath(URDF_ROOT)
    p.setGravity(0, 0, -10)
    return cid

def disconnect():
    p.disconnect()

def create_cylinder_object(x, y, radius, height, color, mass=0.1):
    """
    Create a cylinder at fixed height (z=0.1) so it sits on/near the table,
    with given mass. mass=0 -> static (won't move), mass>0 -> dynamic.
    """
    z = 0.1

    col = p.createCollisionShape(p.GEOM_CYLINDER, radius=radius, height=height)
    vis = p.createVisualShape(p.GEOM_CYLINDER, radius=radius, length=height)
    obj_id = p.createMultiBody(
        baseMass=mass,
        baseCollisionShapeIndex=col,
        baseVisualShapeIndex=vis,
        basePosition=[x, y, z],
    )
    p.changeVisualShape(obj_id, -1, rgbaColor=color)
    return obj_id, np.array([x, y, z])

def create_place_marker():
    """Create a flat, colored square on the table as a visual target."""
    half_extents = [0.05, 0.05, 0.001]
    vis = p.createVisualShape(p.GEOM_BOX, halfExtents=half_extents)

    marker_id = p.createMultiBody(
        baseMass=0.0,
        baseCollisionShapeIndex=-1,
        baseVisualShapeIndex=vis,
        basePosition=[PLACE_POS[0], PLACE_POS[1], 0.001],
    )
    p.changeVisualShape(marker_id, -1, rgbaColor=[1.0, 0.8, 0.2, 1.0])
    return marker_id


def setup_scene(object_type):
    """
    Reset the sim, create plane + table + Panda, a visual place marker,
    and 5 objects (3 bottles, 1 can, 1 glass).
    Exactly one object (matching object_type) is dynamic; others are static.
    """
    p.resetSimulation()
    p.setGravity(0, 0, -10)

    # Ground and table
    p.loadURDF(os.path.join(URDF_ROOT, "plane.urdf"),
               basePosition=[0, 0, -0.65])

    p.loadURDF(os.path.join(URDF_ROOT, "table/table.urdf"),
               basePosition=[0.5, 0, -0.65])

    # Panda arm
    panda_uid = p.loadURDF(os.path.join(URDF_ROOT, "franka_panda/panda.urdf"),
                           useFixedBase=True)

    rest_poses = [0, -0.215, 0, -2.57, 0, 2.356, 2.356, 0.08, 0.08]
    for i in range(7):
        p.resetJointState(panda_uid, i, rest_poses[i])
    p.resetJointState(panda_uid, 9, rest_poses[7])
    p.resetJointState(panda_uid, 10, rest_poses[8])

    # Visual marker for place locatio
    create_place_marker()

    objs_cfg = [
        ("bottle_0", 0.5, 0.03, 0.22, [0.2, 0.4, 0.9, 1.0]),  # bottle 1
        ("bottle_1", 0.6, 0.03, 0.22, [0.2, 0.4, 0.9, 1.0]),  # bottle 2
        ("bottle_2", 0.7, 0.03, 0.22, [0.2, 0.4, 0.9, 1.0]),  # bottle 3
        ("can",      0.8, 0.04, 0.12, [0.7, 0.7, 0.7, 1.0]),  # can
        ("glass",    0.9, 0.025,0.18, [1.0, 1.0, 1.0, 0.5]),  # glass
    ]

    target_uid = None
    target_pos = None

    for name, x, radius, height, color in objs_cfg:
        if object_type == "bottle" and name == "bottle_1":
            mass = 0.1
        elif object_type == "can" and name == "can":
            mass = 0.1
        elif object_type == "glass" and name == "glass":
            mass = 0.1
        else:
            mass = 0.0

        obj_id, pos = create_cylinder_object(
            x=x,
            y=0.0,
            radius=radius,
            height=height,
            color=color,
            mass=mass,
        )

        if mass > 0:
            target_uid = obj_id
            target_pos = pos

    if target_uid is None:
        raise ValueError(f"Did not create a dynamic target for object_type={object_type}")

    return panda_uid, target_uid, target_pos


def get_camera_mats():
    # Main point of this function is to create a side view
    #If we want a more top-down view, will need to change the values here
    view_matrix = p.computeViewMatrixFromYawPitchRoll(
        cameraTargetPosition=[0.6, 0.0, 0.3],
        distance=1.0,
        yaw=0,
        pitch=-15,
        roll=0,
        upAxisIndex=2,
    )
    proj_matrix = p.computeProjectionMatrixFOV(
        fov=50,
        aspect=float(960) / 720,
        nearVal=0.1,
        farVal=3.0,
    )
    return view_matrix, proj_matrix


def capture_frame(view_matrix, proj_matrix, width=1280, height=720):
    _, _, px, _, _ = p.getCameraImage(
        width=width,
        height=height,
        viewMatrix=view_matrix,
        projectionMatrix=proj_matrix,
        renderer=p.ER_BULLET_HARDWARE_OPENGL,
    )
    rgb_array = np.array(px, dtype=np.uint8).reshape(height, width, 4)
    return rgb_array[:, :, :3]


def create_grasp_constraint(panda_uid, obj_id, offset=[0, 0, -0.02]):
    """Attach the object to the gripper"""
    # Get current end-effector pose
    ee_state = p.getLinkState(panda_uid, PANDA_EE_LINK)
    ee_pos = ee_state[0]
    ee_orn = ee_state[1]

    # Get object pose
    obj_state = p.getBasePositionAndOrientation(obj_id)
    obj_pos = obj_state[0]
    obj_orn = obj_state[1]

    # Create a fixed joint between EE link and object base
    constraint_id = p.createConstraint(
        parentBodyUniqueId=panda_uid,
        parentLinkIndex=PANDA_EE_LINK,
        childBodyUniqueId=obj_id,
        childLinkIndex=-1,
        jointType=p.JOINT_FIXED,
        jointAxis=[0, 0, 0],
        parentFramePosition=offset,
        childFramePosition=[0, 0, 0],
        parentFrameOrientation=ee_orn,
        childFrameOrientation=obj_orn,
    )
    return constraint_id


def ee_move_to(panda_uid, target_pos, target_orn, steps=60):
    """
    Move the Panda end-effector to target_pos / target_orn using IK,
    interpolating in joint space over `steps` simulation steps.
    """
    arm_joints = list(range(7))
    current_q = [p.getJointState(panda_uid, j)[0] for j in arm_joints]

    ik_solution = p.calculateInverseKinematics(
        panda_uid,
        PANDA_EE_LINK,
        target_pos,
        target_orn,
        maxNumIterations=80,
    )
    target_q = ik_solution[:7]

    for alpha in np.linspace(0.0, 1.0, steps):
        q = [(1 - alpha) * c + alpha * t for c, t in zip(current_q, target_q)]
        p.setJointMotorControlArray(
            panda_uid,
            arm_joints,
            p.POSITION_CONTROL,
            q,
        )
        p.stepSimulation()
        yield
        current_q = q


def set_fingers(panda_uid, target, steps=30):
    for _ in range(steps):
        p.setJointMotorControl2(panda_uid, 9, p.POSITION_CONTROL, target, force=200)
        p.setJointMotorControl2(panda_uid, 10, p.POSITION_CONTROL, target, force=200)
        p.stepSimulation()
        yield

def open_fingers(panda_uid, frames, view, proj, steps=30):
    i = 0
    for _ in set_fingers(panda_uid, target=0.08, steps=steps):
        if i % 2 == 0:
            frames.append(capture_frame(view, proj))
        i += 1

def close_fingers(panda_uid, frames, view, proj, steps=30):
    i = 0
    for _ in set_fingers(panda_uid, target=0.0, steps=steps):
        if i % 2 == 0:
            frames.append(capture_frame(view, proj))
        i += 1


def run_trial(object_type="bottle", success=True, out_name="bottle_success.mp4"):
    """
    Scripted grasp:
      - move above top of target cylinder
      - move down to just above top
      - close fingers (visual)
      - snap object under gripper + create grasp constraint
      - lift
      - SUCCESS: move to tray and gently place object inside
      - FAILURE: drop object from height

    All 5 objects are present in the scene; only the target is dynamic.
    """
    frames = []

    cid = connect_headless()
    panda_uid, object_uid, obj_pos = setup_scene(object_type)
    view_matrix, proj_matrix = get_camera_mats()

    # Let everything settle
    for i in range(30):
        p.stepSimulation()
        if i % 2 == 0:
            frames.append(capture_frame(view_matrix, proj_matrix))

    # Height of the cylinder
    h = OBJ_HEIGHTS[object_type]

    # Gripper orientation pointing down
    down_orn = p.getQuaternionFromEuler([0., -math.pi, 0.])

    # Positions relative to target cylinder
    top_z = obj_pos[2] + h / 2.0
    above = [obj_pos[0], obj_pos[1], top_z + 0.15]
    grasp = [obj_pos[0], obj_pos[1], top_z + 0.02]
    lift  = [obj_pos[0], obj_pos[1], top_z + 0.25]

    # Place the colored marker
    place_x, place_y, _ = PLACE_POS

    # Use the object's initial center height as reference
    base_z = obj_pos[2]

    place_center = [place_x, place_y, base_z]
    place_above  = [place_x, place_y, base_z + 0.25]
    place_touch  = [place_x, place_y, base_z + 0.02]


    def segment(gen, capture_every=2):
        step_i = 0
        for _ in gen:
            if step_i % capture_every == 0:
                frames.append(capture_frame(view_matrix, proj_matrix))
            step_i += 1

    # Move from rest to "above" object
    segment(ee_move_to(panda_uid, above, down_orn, steps=60))

    # Move down to just above top of cylinder
    segment(ee_move_to(panda_uid, grasp, down_orn, steps=40))

    # Close fingers visually
    close_fingers(panda_uid, frames, view_matrix, proj_matrix, steps=30)

    # Attach object under gripper via constraint, snapping it upright
    ee_state = p.getLinkState(panda_uid, PANDA_EE_LINK)
    ee_pos = ee_state[0]

    desired_top_z = ee_pos[2] - 0.02
    desired_center_z = desired_top_z - h / 2.0
    p.resetBasePositionAndOrientation(
        object_uid,
        [ee_pos[0], ee_pos[1], desired_center_z],
        [0, 0, 0, 1],
    )
    grasp_constraint_id = create_grasp_constraint(panda_uid, object_uid, offset=[0, 0, -0.02])

    # Lift the grasped object
    segment(ee_move_to(panda_uid, lift, down_orn, steps=60))

    if success:
        # Move horizontally to the tray, above the final place height
        segment(ee_move_to(panda_uid, place_above, down_orn, steps=60))

        # Move down to just above the tray floor (object center ~0.1)
        segment(ee_move_to(panda_uid, place_touch, down_orn, steps=40))

        # Release: remove constraint and open fingers
        if grasp_constraint_id is not None:
            p.removeConstraint(grasp_constraint_id)
            grasp_constraint_id = None
        open_fingers(panda_uid, frames, view_matrix, proj_matrix, steps=30)

        # Short pause
        for i in range(30):
            p.stepSimulation()
            if i % 2 == 0:
                frames.append(capture_frame(view_matrix, proj_matrix))
    else:
        # Failure: drop the object from the lift pose
        if grasp_constraint_id is not None:
            p.removeConstraint(grasp_constraint_id)
            grasp_constraint_id = None

        # Open fingers as it falls
        open_fingers(panda_uid, frames, view_matrix, proj_matrix, steps=30)

        # Let it fall and bounce
        for i in range(60):
            p.stepSimulation()
            if i % 2 == 0:
                frames.append(capture_frame(view_matrix, proj_matrix))

    disconnect()

    out_path = OUT_DIR / out_name
    imageio.mimsave(out_path, frames, fps=FPS)
    print("Saved:", out_path)


In [ ]:
run_trial("bottle", success=True,  out_name="bottle_success_V9.mp4")
run_trial("can",    success=True,  out_name="can_success_V9.mp4")
run_trial("glass",  success=True,  out_name="glass_success_V9.mp4")
run_trial("glass",  success=False, out_name="glass_failure_drop_V9.mp4")

Saved: videos/bottle_success_V9.mp4
Saved: videos/can_success_V9.mp4
Saved: videos/glass_success_V9.mp4
Saved: videos/glass_failure_drop_V9.mp4


In [ ]:
for f in os.listdir("/content/videos"):
    print(f)

bottle_success_V6.mp4
bottle_success_V4.mp4
bottle_success_V5.mp4
bottle_success_newV3.mp4


In [ ]:
from google.colab import files
files.download("/content/videos/glass_failure_drop_V9.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>